In [1]:
# hierarchical bert with linear layer model  both bert and linear layers are trained
# 5*bert + linear layer 5*768-50-sigmoid  lr=0.00001,no weights in loss function, no bias in linear layer       validation loss decrease to 44.3, validation f1 increase to 0.74  early stop at 11th/12th epoch
# 5*bert + linear layer 5*768-50-sigmoid  lr=0.00001, weights in loss function, no bias in linear layer       validation loss decrease to 44.8, validation f1 increase to 0.73  early stop at 9th/10th epoch
# 5*bert + linear layer 5*768-512-Gelu-50-sigmoid  lr=0.00001, no weights in loss function, no bias in linear layer       validation loss decrease to 45.0, validation f1 increase to 0.75  early stop at 10th epoch
# # 5*bert + linear layer 5*768-512-Gelu-50-sigmoid  lr=0.00001, weights in loss function, no bias in linear layer       validation loss decrease to 45.0, validation f1 increase to 0.74  early stop at 10th epoch

In [1]:
import pandas as pd
import re
from tqdm import tqdm
import torch
import torchmetrics
import ast
import random
import transformers
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
# import wandb

/u/home/niur/venv_1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/home/niur/venv_1/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
def text_cleaning(dataframe):
    for i in range(dataframe.shape[0]):
        dataframe['TEXT'].iloc[i] = re.sub("(Admission Date:)|(Discharge Date:)|(Service:)|(Date of Birth:)|(Sex:)|(Attending:)|(Provider:)|(Name:)|(Date/Time:)|(MD Phone:)|(Completed by:)|(Job#:)|(Dictated By:)", "", dataframe['TEXT'].iloc[i]) 
        dataframe['TEXT'].iloc[i] = re.sub(r'\[\*\*.*?\*\*\]', '', dataframe['TEXT'].iloc[i])  # delete [**      **]
        dataframe['TEXT'].iloc[i] = re.sub('[0-9]{2}:[0-9]{2}', '', dataframe['TEXT'].iloc[i])  # delete 12:11
        dataframe['TEXT'].iloc[i] = re.sub('[0-9]{2}:[0-9]{2}AM', '', dataframe['TEXT'].iloc[i])  # delete 12:11AM
        dataframe['TEXT'].iloc[i] = re.sub('[0-9]{2}:[0-9]{2}PM', '', dataframe['TEXT'].iloc[i])  # delete 12:11PM
        dataframe['TEXT'].iloc[i] = re.sub('==+', ' ', dataframe['TEXT'].iloc[i])   # delete redundant space ' '
        dataframe['TEXT'].iloc[i] = re.sub(' +', ' ', dataframe['TEXT'].iloc[i])   # delete redundant space ' '
        dataframe['TEXT'].iloc[i] = re.sub('\n', ' ', dataframe['TEXT'].iloc[i]) # change \n to ' '
        dataframe['TEXT'].iloc[i] = re.sub('# *', ' ', dataframe['TEXT'].iloc[i])
        # dataframe['TEXT'].iloc[i] = re.sub(' - ', ' ', dataframe['TEXT'].iloc[i])
        # dataframe['TEXT'].iloc[i] = re.sub('- ', ' ', dataframe['TEXT'].iloc[i])
        # dataframe['TEXT'].iloc[i] = re.sub(': *', ' ', dataframe['TEXT'].iloc[i])
        # dataframe['TEXT'].iloc[i] = re.sub('[0-9]\. ', ' ', dataframe['TEXT'].iloc[i])
        # dataframe['TEXT'].iloc[i] = re.sub('\* ', ' ', dataframe['TEXT'].iloc[i])
    return dataframe

In [4]:
def convert_icdstr_to_list(df):
    all_icds = []
    for i in list(df['ICD9_CODE']):
        all_icds.append(ast.literal_eval(''.join(i)))
    df['ICD9_CODE'] = all_icds
    return df

In [5]:
def create_all_icd_labels(datafram):
    disease_list = list(pd.read_csv('./data/ICD9_description_LONG_SHORT_diagnose.csv')['DIAGNOSIS CODE'])
    label_list = []
    for i in range(datafram.shape[0]):
        label = torch.tensor([0]*14567, dtype=int)
        for token in list(datafram['ICD9_CODE'].iloc[i]):
            index = disease_list.index(token)
            label[index] = int(1)
        label_list.append(label)
    label_tensor = torch.stack(label_list)

    return label_tensor

In [6]:
def create_text_tokens(datafram, tokenizer):

    input_ids_list = []

    for i in range(datafram.shape[0]):

        tokenizer_outputs = tokenizer(datafram['TEXT'].iloc[i], padding='max_length' ,truncation=True, max_length=4*512, return_tensors='pt')
        bert_token_tensor = tokenizer_outputs['input_ids'].reshape((4,512))
        bert_token_tensor[:,0] = 101
        if bert_token_tensor.shape[0] > 1:
            bert_token_tensor[:-1,-1] = 102

        input_ids_list.append(bert_token_tensor)

    input_ids_tensor = torch.stack(input_ids_list, dim=0)

    return input_ids_tensor

In [2]:
df_train_top500 = pd.read_csv('./data/top500_datasets/trainset_top500_new.csv')
df_validation_top500 = pd.read_csv('./data/top500_datasets/validationset_top500_new.csv')

In [8]:
df_train_top500 = text_cleaning(df_train_top500)
df_validation_top500 = text_cleaning(df_validation_top500)

df_train_top500 = convert_icdstr_to_list(df_train_top500)
df_validation_top500 = convert_icdstr_to_list(df_validation_top500)

label_tensor_train = create_all_icd_labels(df_train_top500)
label_tensor_validation = create_all_icd_labels(df_validation_top500)

tokenizer = transformers.BertTokenizer.from_pretrained('./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b', ignore_mismatched_sizes=True)



bert_token_tensor_train = create_text_tokens(df_train_top500, tokenizer)
bert_token_tensor_validation = create_text_tokens(df_validation_top500, tokenizer)



/u/home/niur/htc_mimic3/ipykernel_1146530/246453972.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['TEXT'].iloc[i] = re.sub("(Admission Date:)|(Discharge Date:)|(Service:)|(Date of Birth:)|(Sex:)|(Attending:)|(Provider:)|(Name:)|(Date/Time:)|(MD Phone:)|(Completed by:)|(Job#:)|(Dictated By:)", "", dataframe['TEXT'].iloc[i])
/u/home/niur/htc_mimic3/ipykernel_1146530/246453972.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['TEXT'].iloc[i] = re.sub(r'\[\*\*.*?\*\*\]', '', dataframe['TEXT'].iloc[i])  # delete [**      **]
/u/home/niur/htc_mimic3/ipykernel_1146530/246453972.py:5: SettingWithCop

In [9]:
print(label_tensor_train.shape)
print(label_tensor_validation.shape)
print(bert_token_tensor_train.shape)
print(bert_token_tensor_validation.shape)

torch.Size([8496, 14567])
torch.Size([1500, 14567])
torch.Size([8496, 4, 512])
torch.Size([1500, 4, 512])


In [10]:
class CreateDataset(Dataset):
    def __init__(self, label_tensor, bert_token_tensor):
        self.label_tensor = label_tensor
        self.bert_token_tensor = bert_token_tensor

    def __len__(self):
        return self.label_tensor.shape[0]

    def __getitem__(self, index):
        return self.label_tensor[index], self.bert_token_tensor[index]

In [11]:
training_dataset_top500 = CreateDataset(label_tensor=label_tensor_train, bert_token_tensor=bert_token_tensor_train)
validation_dataset_top500 = CreateDataset(label_tensor=label_tensor_validation, bert_token_tensor=bert_token_tensor_validation)


batch_size = 8

trainset = DataLoader(dataset=training_dataset_top500, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=8)
validationset = DataLoader(dataset=validation_dataset_top500, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=8)

In [12]:
class HIBERT_with_Linear(torch.torch.nn.Module):
    def __init__(self):
        super(HIBERT_with_Linear, self).__init__()
        self.clinical_DS_bert =  transformers.BertModel.from_pretrained('./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b')
        
        self.linear1 = torch.nn.Linear(4*768, 6*768, bias=True)
        self.activ1 = torch.nn.GELU()
        self.linear2 = torch.nn.Linear(6*768, 8*768, bias=True)
        self.activ2 = torch.nn.GELU()
        self.linear3 = torch.nn.Linear(8*768, 10*768, bias=True)
        self.activ3 = torch.nn.GELU()
        self.linear4 = torch.nn.Linear(10*768, 14567, bias=False)

    def forward(self, tokens):

        templist = []
        for item in tokens:   # bs*4*512
            single_output = self.clinical_DS_bert(item).last_hidden_state[:,0,:].reshape((1,4*768))  # (1,4*768)
            templist.append(single_output)   # [(1,4*768) , (1,4*768) ......]
        x = torch.cat(templist, dim=0)   # (bs, 4*768)

        x = self.linear1(x)
        x = self.activ1(x)
        x = self.linear2(x)
        x = self.activ2(x)
        x = self.linear3(x)
        x = self.activ3(x)
        x = self.linear4(x)
        
        return x

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = HIBERT_with_Linear()
model = model.to(device)

LR = 1e-6
criterion = torch.nn.BCELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

Some weights of the model checkpoint at ./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
def valid(validationset):
    model.eval()
    print('Validation...')
    
    with torch.no_grad():
        loss_validation = 0
        predictions_AUROC = torch.tensor([],dtype=float).to(device)
        predictions_f1 = torch.tensor([], dtype=int).to(device)
        labels_0_1 = torch.tensor([], dtype=int).to(device)

        for i, (label_tensor_validation, bert_token_tensor_validation) in enumerate(validationset):
            label_tensor_validation, bert_token_tensor_validation = label_tensor_validation.to(device), bert_token_tensor_validation.to(device)
            outputs = model(bert_token_tensor_validation)  # (bs,14567)
            predictions_AUROC = torch.cat([predictions_AUROC, outputs])
            
            outputs = torch.nn.Sigmoid()(outputs)

            valid_loss = criterion(outputs.float(), label_tensor_validation.float())
            loss_validation += valid_loss.item()

            outputs = torch.gt(outputs, 0.5)*int(1)
            predictions_f1 = torch.cat([predictions_f1, outputs])
            labels_0_1 = torch.cat([labels_0_1, label_tensor_validation])
        
        
        valid_f1_micro = torchmetrics.functional.classification.multilabel_f1_score(predictions_f1, labels_0_1, num_labels=14567, threshold=0.5, average='micro', multidim_average='global')
        valid_f1_weighted = torchmetrics.functional.classification.multilabel_f1_score(predictions_f1, labels_0_1, num_labels=14567, threshold=0.5, average='weighted', multidim_average='global')
        valid_AUROC_micro = torchmetrics.functional.classification.multilabel_auroc(predictions_AUROC, labels_0_1, num_labels=14567, average='micro')
        valid_AUROC_weighted = torchmetrics.functional.classification.multilabel_auroc(predictions_AUROC, labels_0_1, num_labels=14567, average='weighted')

        valid_precision = torchmetrics.functional.classification.multilabel_precision(predictions_f1, labels_0_1, num_labels=14567,multidim_average='global', average='micro')
        valid_recall = torchmetrics.functional.classification.multilabel_recall(predictions_f1, labels_0_1, num_labels=14567,multidim_average='global', average='micro')
        valid_true_positive = torch.sum((predictions_f1+labels_0_1)==2)/labels_0_1.shape[0]/labels_0_1.shape[1]
        valid_true_negative = torch.sum((predictions_f1+labels_0_1)==0)/labels_0_1.shape[0]/labels_0_1.shape[1]


    
    return loss_validation/df_validation_top500.shape[0], valid_f1_micro, valid_f1_weighted, valid_AUROC_micro, valid_AUROC_weighted, valid_precision, valid_recall, valid_true_positive, valid_true_negative
    
        
def start_training(trainset, validationset, epochs=75):
    # config = { 'batch_size': batch_size, 'lr':LR, 'loss':'BCE', 'optim':'Adam', 'comment':'4 layers of FCN'}
    # wandb.init(project="FCN_baseline", entity="htc-mimic3", config=config)
    
    print('Training...')

    for epoch in range(epochs):
        print(f'Epoch: {epoch+1}')
        #TRAININING STEP

        loss_training = 0.0
        model.train()
        for i, (label_tensor_train, bert_token_tensor_train) in enumerate(trainset):
            label_tensor_train, bert_token_tensor_train = label_tensor_train.to(device), bert_token_tensor_train.to(device)
            outputs = model(bert_token_tensor_train)  # bs*14567
            outputs = torch.nn.Sigmoid()(outputs)

            loss = criterion(outputs.float(), label_tensor_train.float())  # (bs*14567)  (bs*14567)

            
            #backward step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()         
            loss_training += loss.item()

        
        print('the training loss in epoch',epoch+1,'is:',loss_training/df_train_top500.shape[0])
        loss_validation, valid_f1_micro, valid_f1_weighted, valid_AUROC_micro, valid_AUROC_weighted, valid_precision, valid_recall, valid_true_positive, valid_true_negative = valid(validationset=validationset)
        
        print('the loss_validation in epoch',epoch+1,'is:',loss_validation)
        print('the valid_f1_micro in epoch',epoch+1,'is:',valid_f1_micro)
        print('the valid_f1_weighted in epoch',epoch+1,'is:',valid_f1_weighted)
        print('the valid_AUROC_micro in epoch',epoch+1,'is:',valid_AUROC_micro)
        print('the valid_AUROC_weighted in epoch',epoch+1,'is:',valid_AUROC_weighted)
        print('the valid_precision in epoch',epoch+1,'is:',valid_precision)
        print('the valid_recall in epoch',epoch+1,'is:',valid_recall)
        print('the valid_true_positive in epoch',epoch+1,'is:',valid_true_positive)
        print('the valid_true_negative in epoch',epoch+1,'is:',valid_true_negative)

        # wandb.log({'training_loss': loss_training/df_train_top500.shape[0]})
        # wandb.log({'loss_validation': loss_validation})
        # wandb.log({'valid_f1_micro': valid_f1_micro})
        # wandb.log({'valid_f1_weighted': valid_f1_weighted})
        # wandb.log({'valid_AUROC_micro': valid_AUROC_micro})
        # wandb.log({'valid_AUROC_weighted': valid_AUROC_weighted})
        # wandb.log({'valid_precision': valid_precision})
        # wandb.log({'valid_recall': valid_recall})
        # wandb.log({'valid_true_positive': valid_true_positive})
        # wandb.log({'valid_true_negative': valid_true_negative})



        print('EPOCH DONE!')
    print('Finished Training')


In [15]:
start_training(trainset=trainset, validationset=validationset)

Training...
Epoch: 1


KeyboardInterrupt: 